In [ ]:
!pip uninstall -y eonacs && pip install  git+https://github.com/njnmco/smc #--log /dev/stderr
# https://stackoverflow.com/questions/67798070/raytune-is-throwing-error-module-pickle-has-no-attribute-picklebuffer-whe
# downgrade pickle5 for ray
!pip install datasets ray[tune] pickle5==0.0.10 pacmap
# NB restart kernel after setup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/njnmco/smc to /tmp/pip-req-build-nan2xo42
  Running command git clone -q https://github.com/njnmco/smc /tmp/pip-req-build-nan2xo42
     |████████████████████████████████| 4.2 MB 20.5 MB/s 
     |████████████████████████████████| 646 kB 44.2 MB/s 
     |████████████████████████████████| 596 kB 50.1 MB/s 
     |████████████████████████████████| 6.6 MB 52.7 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
  Created wheel for eonacs: filename=eonacs-1.0.0-py3-none-any.whl size=5532 sha256=b0c0aa2a054569957213529dcb509d6857c9b18bae1b6ef3a5d4c141cf23ce76
  Stored in directory: /tmp/pip-ephem-wheel-cache-1l8f3v2k/wheels/6e/02/6d/470a51584ecf5233c1da27aab5d5e137df8b9ab9cc345b2a49
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391586 sha256=fcb5c02faef51faa0dc55b78da7456a446e8a5caf5795794e9ae683b3e11ec83
  Stored in direc

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pacmap

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
from eonacs.common.bloom import bloom_categories
from eonacs.common.nlp import hashtrick

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Created from tune.py
best = {'lr': 0.6449861432586895, 'n_buckets': 556.0, 'n_components': 76.0, 'n_far': 56.0, 'n_mid': 41.0, 'n_near': 48.0}

In [ ]:
import eonacs.common.util as util
util.colab_map_drive()

Mounted at /content/drive


In [ ]:
!ls /content/drive/

MyDrive  Shareddrives


In [ ]:
tasks, hyper_e = pd.read_pickle("/content/drive/MyDrive/smc/data/tasks_dbert_hyper_e_refactor.pkl.gz")

In [ ]:
tasks.head()

,text
0,Direct or coordinate an organization's financi...
1,Appoint department heads or managers and assig...
2,Analyze operations to evaluate performance of ...
3,"Direct, plan, or implement policies, objective..."
4,"Prepare budgets for approval, including those ..."


In [ ]:
    big = \
        np.concatenate( [ hyper_e,
                          bloom_categories(tasks["text"]),
                          hashtrick(tasks["text"], int(best["n_buckets"])) ],
                       axis=1)
    big = np.asarray(big)

In [ ]:
%%time
model = pacmap.PaCMAP(n_components = int(best["n_components"]),
        n_neighbors  = int(best["n_near"]),
        MN_ratio = best["n_mid"] / best["n_near"],
        FP_ratio = best["n_far"] / best["n_near"],
        lr = best["lr"], save_tree=True)
pac_e = model.fit_transform(big)

CPU times: user 9min 1s, sys: 2.19 s, total: 9min 3s
Wall time: 8min 47s


In [ ]:
pd.to_pickle((tasks, hyper_e, pac_e), "/content/drive/MyDrive/smc/data/tasks_pac_e.pkl.gz")